In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

PCA算法相关的大部分知识以及代码实现和样例。

### 1.1 什么是主成分分析
在多元统计分析中，主成分分析（Principal components analysis，PCA）是一种统计分析、简化数据集的方法。它利用正交变换来对一系列可能相关的变量的观测值进行线性变换，从而投影为一系列线性不相关变量的值，这些不相关变量称为主成分（Principal Components）。

基本思想：
- 将坐标轴中心移到数据的中心，然后旋转坐标轴，使得数据在C1轴上的方差最大，即全部数据个体在该方向上的投影最为分散。意味着更多的信息被保留下来。C1成为第一主成分。
- C2第二主成分：找一个C2，使得C2与C1的协方差（相关系数）为0，以免与C1信息重叠，并且使数据在该方向的方差尽量最大。以此类推，找到第三主成分，第四主成分......第p个主成分。p个随机变量可以有p个主成分。

接下来我们根据主成分分析的算法步骤一步一步、一点一点地来分析，为什么这么做。
### 1.2主成分分析算法
> 输入：数据集$D=$ {$\vec{x_1}, \vec{x_2}, ..., \vec{x_m}$}，m个样本，n 个特征，目标低维空间维数 k
> 1. 对所有样本进行中心化
> 2. 计算样本的协方差矩阵
> 3. 对协方差矩阵做特征值分解得到特征值和特征向量
> 4. 取最大的 k 个特征值对应的特征向量构成$k\times n$投影矩阵$W^*$
> 5. 对每一个样本$x_i$作变换，$z_i^T=W^*x_i^T$
>
> 输出：降维后的数据集$D'=$ {$\vec{z_1}, \vec{z_2}, ..., \vec{z_m}$}

### 2.1 什么是中心化，为什么中心化？
中心化也就是一个去平均值的操作，即每一个样本数据减去它对应数据的平均值。

原因很简单，就是为了更好地找到一个坐标轴（中心化后的坐标轴原点就在均值 0 处），使得数据在该坐标轴上的投影最为分散或者说方差最大。这样我们就尽可能地保留了多的信息而完成了数据降维。

In [2]:
x = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
print(x)
x = x - np.mean(x, axis=0)
print(x)

[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]]
[[-4. -4.]
 [-2. -2.]
 [ 0.  0.]
 [ 2.  2.]
 [ 4.  4.]]


### 2.2 协方差矩阵
在统计学中，方差是用来度量单个随机变量的离散程度，而协方差则一般用来刻画两个随机变量的相似程度。

方差：
$\sigma_x^2=\displaystyle \sum^{n}_{i=1}{\frac{(x_i - \bar{x})^2}{n - 1}}$

这里的方差是样本方差，除以的是 n - 1 。n 是样本总数，$\bar{x}$ 是样本均值。

协方差：
$\sigma_{(x, y)}=\displaystyle \sum^{n}_{i=1}{\frac{(x_i - \bar{x})(y_i - \bar{y})}{n - 1}}$

这是两个变量$x$，$y$的协方差。$\bar{x}$，$\bar{y}$分别是两个变量的均值。

**可以看到方差是协方差的特殊情况**

假设有两个变量$x=${$x_1, x_2, ..., x_m$}，$y=${$y_1, y_2, ..., y_m$}，构建它们之间的$2\times2$协方差矩阵。

协方差矩阵：$\Sigma=\begin{bmatrix}\sigma_{(x, x)} & \sigma_{(x, y)} \\ \sigma_{(y, x)} & \sigma_{(y, y)}\end{bmatrix}$

其中，$\sigma_{(x, x)}$是 $x$ 的方差，$\sigma_{(x, y)}$是 $x$和$y$的协方差。

推广到 n 个变量，我们就可以构建一个$n\times n$的协方差矩阵，对角线元素就是对应的各个随机变量的方差，非对角线元素就是对应变量的协方差。并且我们可以知道协方差矩阵是一个对称矩阵。

In [3]:
d = np.random.randn(35).reshape(7, 5)
d_conv = np.cov(d, rowvar=False)  # 设为False，将每一列看作一个变量
d, d_conv

(array([[-0.48411697,  0.50717112,  0.36416339, -0.80854973,  0.60034001],
        [-0.74012045, -1.57750774,  1.21821849, -0.17107428, -0.00774912],
        [-1.52210758,  1.3720564 , -0.65758342,  0.08713715, -0.10595215],
        [ 0.36526404,  0.20023598,  0.27540064,  0.03561132, -0.01609642],
        [ 1.63223259, -0.19379633, -1.0737328 ,  0.26232791,  0.84171532],
        [ 0.11479205, -0.21921563, -0.12423041, -0.61867203, -0.37441627],
        [-0.22304756, -0.50008244, -0.40050632, -0.97541677, -0.48734953]]),
 array([[ 0.9757568 , -0.22892447, -0.28378257,  0.11752756,  0.22754144],
        [-0.22892447,  0.82998892, -0.36198992,  0.07175188,  0.05953912],
        [-0.28378257, -0.36198992,  0.57183557, -0.08150307, -0.06033744],
        [ 0.11752756,  0.07175188, -0.08150307,  0.23509747,  0.09579706],
        [ 0.22754144,  0.05953912, -0.06033744,  0.09579706,  0.23819051]]))

### 2.3 特征值分解和奇异值分解
##### 2.3.1 特征值与特征向量
给出一个$n\times n$**方阵**$A$，如果数$\lambda$和$n$维非零向量$x$使得

$$Ax=\lambda x$$

成立。那么将$\lambda$称为方阵$A$的特征值，$x$称为它的特征向量。

将上式右边移到左边，提公因式得到

$$(A-\lambda I)x=0$$

其中$I$是$n$阶单位阵。这就是一个关于$x$的$n$个未知数，$n$个方程的齐次线性方程组，存在非零解的充分必要条件是系数矩阵$(A-\lambda I)$的秩小于它的阶数即可推出系数行列式的值为零

$$|A-\lambda I|=0\Rightarrow \begin{vmatrix} a_{11}-\lambda & a_{12} & \cdots & a_{1n} \\ a_{21} & a_{22}-\lambda & \cdots & a_{2n} \\ \vdots & \vdots &\ddots & \vdots \\ a_{n1} & a_{n2} & \cdots & a_{nn}-\lambda \end{vmatrix}=0$$

这是关于特征值$\lambda$的一元$n$次方程，称为方阵$A$的特征方程。特征方程在复数范围内恒有解，个数是方程的次数，也就是方阵$A$的阶数，重根按重数计算，因此$n$阶方阵$A$在复数范围内有$n$个特征值。

假设$\lambda_i$是解出来的一个特征值，将其代入到

$$(A-\lambda_i I)x=0$$

就可求得非零解$x=p_i$，那么$p_i$就是$A$的对应于特征值$\lambda_i$的特征向量。注意：若$p_i$是$A$的对应于特征值$\lambda_i$的特征向量，那么$kp_i(k\neq0)$也是$A$对应于$\lambda_i$的特征向量。


##### 2.3.2 特征值分解（EVD）
特征值分解就是将矩阵分解成特征值和特征向量，用特征值和特征向量来表示原矩阵。这里的$x_i$是对应特征值的特征向量的列向量，根据特征值和特征向量的定义

$$Ax=\lambda x \Rightarrow A(x_1, x_2, ..., x_n)=(\lambda_1 x_1, \lambda_2 x_2, ..., \lambda_n x_n)$$

$$\Rightarrow(\lambda_1 x_1, \lambda_2 x_2, ..., \lambda_n x_n)=(x_1, x_2, ..., x_n)\begin{bmatrix}\lambda_1 & 0 & \cdots & 0 \\ 0 & \lambda_2 & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \cdots & \lambda_n\end{bmatrix}$$

假设特征向量组线性无关，那么$X$可逆，$X$是特征向量按列组成的矩阵，设$\begin{bmatrix}\lambda_1 & 0 & \cdots & 0 \\ 0 & \lambda_2 & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \cdots & \lambda_n\end{bmatrix}$为$\Lambda$，就有

$$AX=X\Lambda \Rightarrow A=X\Lambda X^{-1}$$

至此，我们用特征值和特征向量成功表示了矩阵$A$，再通过计算特征值和对应的特征向量，那么就完成了特征值分解。

In [4]:
A = np.array([[1, 2, 8], [-1, 5, 10], [0, 2, 9]])
l, x = np.linalg.eig(A)
# 计算方阵 A 的特征值和右特征向量。右特征向量就是我们一直在讨论的，左特征向量类似，有兴趣可以去查一下
# 我心想我们手算特征值和特征向量是一回事，但计算机程序计算应该又是另一回事。就想去查一下具体程序是怎么计算的
# 查了numpy文档，文档说是用的 LAPACK 计算，为此我又去查了 LAPACK 库，找到了源码，但是我看不懂，感兴趣的同学可以去浏览一下，读一读
# LAPACK链接：https://netlib.org/lapack/

print('特征值:\n', l)
print('特征向量组成的矩阵:\n', x)

# 根据公式验证结果
l = np.array([[l[0], 0, 0], [0, l[1], 0], [0, 0, l[2]]])  # 特征值放对角线
x_I = np.linalg.inv(x)  # 计算 numpy 数组的逆
A_1 = np.linalg.multi_dot([x, l, x_I])  # 按顺序计算列表内三个元素的内积
print(np.allclose(A, A_1))  # allclose(): 如果两个数组在误差允许范围之内元素相等，返回True


特征值:
 [ 1.          2.30958424 11.69041576]
特征向量组成的矩阵:
 [[ 0.82416338 -0.27534092  0.51234347]
 [ 0.54944226  0.92107261  0.68920848]
 [-0.13736056 -0.27534092  0.51234347]]
True


##### 2.3.3 奇异值分解（SVD）
相对于特征值分解只适用于方阵，奇异值分解适合所有实数矩阵。设$m\times n$实矩阵$A$，可将其分解为

$$A=U\Sigma V^T$$

（这个式子是怎么来的，我也不知道）其中，$U$是满足$U^TU=I$的$m$阶酉矩阵，$U$是满足$V^TV=I$的$n$阶酉矩阵。$\Sigma$是$m\times n$的矩阵，其中$(\Sigma)_{ii}=\sigma_i$，其他位置元素均为 0 ，$\sigma_i$均为非负实数且满足$\sigma_1 \geq \sigma_2 \geq \cdots \geq 0$

> 酉矩阵：
> 
> 若矩阵$A$满足：$A^HA=AA^H=I$，则称矩阵$A$是一个酉矩阵
> 
> 其中$A^H$表示矩阵$A$的共轭转置，矩阵$A$的共轭转置就是取$A$中元素的共轭复数（实部相同，虚部相反的两个复数互为共轭复数）构成的矩阵$\bar A$的转置，即：$A^H=(\bar A)^T$

其中$U$的列向量称为矩阵$A$的左奇异向量，$V$的列向量称为矩阵$A$的右奇异向量。$\sigma_i$就是矩阵$A$的奇异值，其个数等于$A$的秩。

接下来我们看如何计算$U, \Sigma, V$

首先将要分解的$m\times n$的矩阵$A$乘上它的转置，我们就可以得到一个$m\times m$的方阵$(AA^T)$。那么是方阵的话我们就可以对其进行特征值分解，得到
$$(AA^T)U=\lambda_i U$$
这里的$U$就是我们需要的那个$U$

同上一步，接下来将$A$的转置乘上它本身，得到一个$n\times n$的方阵$(A^TA)$。同样进行特征值分解，得到

$$(A^TA)V=\lambda_i V$$
就可计算得到$V$

由$A=U\Sigma V^T \Rightarrow AV=U\Sigma V^TV \Rightarrow AV=U\Sigma \Rightarrow \begin{bmatrix}a_{11} &  \cdots & a_{1n} \\ \vdots &  \ddots & \vdots \\ a_{m1} & \cdots & a_{mn} \end{bmatrix}(v_1, v_2, ..., v_n)=(u_1, u_2, ..., u_m)\Sigma$

通过最后得出矩阵方程我们就可以解出$\sigma_i$，至此我们对$A$的奇异值分解就结束了。

In [5]:
A = np.array([[1, 2, 8], [-1, 5, 10], [0, 2, 9], [2, 1, 7], [0, 3, 10]])  # shape: (5, 3)
s = np.linalg.svd(A)  # 返回包含三个元素的tuple, 依次是U、Sigma和V^T
'''
(array([[-0.39725362, -0.28034428,  0.20685827, -0.83712199, -0.14153824],
        [-0.52944196,  0.68764079,  0.41136745,  0.16104935, -0.22733515],
        [-0.44192138, -0.11186842, -0.71914378,  0.15421894, -0.50123816],
        [-0.33830158, -0.65916459,  0.4473559 ,  0.49908567, -0.04289846],
        [-0.50213375,  0.039303  , -0.26587853,  0.02049123,  0.821709  ]]),
 array([20.78369486,  3.16909449,  0.99743079]),
 array([[-0.02619432, -0.29687997, -0.95455547],
        [-0.72144086,  0.66660144, -0.18752494],
        [ 0.69198045,  0.68374323, -0.23164251]]))
'''
# 根据公式验证结果
Sigma = np.array([[s[1][0], 0, 0], [0, s[1][1], 0], [0, 0, s[1][2]], [0, 0, 0], [0, 0, 0]])  # Sigma形状是(m, n), 即(5, 3)
A_ = np.linalg.multi_dot([s[0], Sigma, s[2]])
print(np.allclose(A, A_))

True


### 3.1 PCA算法推导
https://www.cnblogs.com/pinard/p/6239403.html

虽然在PCA算法中求得协方差矩阵的特征值和特征向量的方法是特征值分解，但在算法的实现上，使用SVD来求得协方差矩阵特征值和特征向量会更高效。sklearn库中的PCA算法就是利用SVD实现的。

接下来我们自己编写代码实现PCA算法。

### 3.2 代码实现
先后使用numpy和sklearn进行实现，最后比较我们自己写的和封装好的库有什么不同。

数据集我选的是kaggle游乐场的表格数据，刚好这个数据较小，只有6个特征，400多个样本，非常适合做实验。

如果有看不懂代码的话没关系，我们现在的重点是搞清PCA算法。

In [6]:
data = pd.read_csv(r'D:\Users\my_projects\vscode_projects\kaggle\Kaggle-playground\1\train.csv')
display(data.head(10))
print('The shape of dataset: ', data.shape)

,id,gravity,ph,osmo,cond,urea,calc,target
0,0,1.013,6.19,443,14.8,124,1.45,0
1,1,1.025,5.40,703,23.6,394,4.18,0
2,2,1.009,6.13,371,24.5,159,9.04,0
3,3,1.021,4.91,442,20.8,398,6.63,1
4,4,1.021,5.53,874,17.8,385,2.21,1
5,5,1.025,6.90,947,28.4,395,2.64,1
6,6,1.008,5.09,371,15.5,159,2.17,1
7,7,1.015,5.53,450,8.1,170,1.16,0
8,8,1.025,7.38,736,25.3,418,1.52,0
9,9,1.017,6.61,527,20.0,75,4.49,1


The shape of dataset:  (414, 8)


In [7]:
data = data.drop(columns=['id', 'target'])  # id 和 label 是我们不需要的
data_npy = np.array(data)

data_cov = np.cov(data_npy, rowvar=False)  # 将列也就是样本的特征看作变量，计算它们的协方差，我们得到的应该是一个 6 * 6 的矩阵
# 对协方差矩阵进行特征值分解
eigenvalue, eigenvector = np.linalg.eig(data_cov)  # eigenvalue: 特征值，eigenvector: 特征向量
print(eigenvalue,'\n', eigenvector)
# 将特征值排序
sorted_id = sorted(range(len(eigenvalue)), key=lambda k: eigenvalue[k], reverse=True)  # 返回降序排列好的特征值对应的索引
# 假设我们降到 2 维，即取最前面的两个特征向量
w = np.array([eigenvector[sorted_id[0]], eigenvector[sorted_id[1]]])
# 将每个样本转换
data_1 = np.matmul(w, data_npy.T).T

[6.85715156e+04 5.15004584e+03 2.71968013e+01 7.68800793e+00
 2.11078692e-05 3.69411991e-01] 
 [[-1.79063909e-05  2.08690320e-06 -6.15156649e-06 -4.06901784e-04
   9.99999695e-01 -6.67028152e-04]
 [ 7.68638715e-04 -5.01553622e-05  3.99833662e-03  1.70683061e-02
   6.73909096e-04  9.99845807e-01]
 [-8.87259223e-01 -4.60387878e-01 -2.84272061e-02  2.33005260e-03
  -1.36647310e-05  7.32904573e-04]
 [-1.96225341e-02 -2.37674450e-02  9.99226977e-01  2.40046859e-02
   1.26831684e-05 -4.39176029e-03]
 [-4.60813082e-01  8.87384618e-01  1.18746420e-02  7.66725471e-03
  -6.76350774e-06  2.20398548e-04]
 [-6.06126307e-03  5.16209488e-03  2.40904140e-02 -9.99533925e-01
  -3.95362683e-04  1.69718306e-02]]


In [8]:
skPCA = PCA(n_components=2, copy=True)
skPCA.fit(data_npy)
print(skPCA.explained_variance_)
print(skPCA.components_)

data_2 = skPCA.transform(data_npy)

[68571.5156409   5150.04583903]
[[ 1.79063909e-05 -7.68638715e-04  8.87259223e-01  1.96225341e-02
   4.60813082e-01  6.06126307e-03]
 [ 2.08690320e-06 -5.01553622e-05 -4.60387878e-01 -2.37674450e-02
   8.87384618e-01  5.16209488e-03]]


通过比较我们纯手打的 numpy 代码的结果和两行就搞定的 sklearn 代码结果发现，特征值是一模一样（最后两个的顺序不一样），特征向量不同没关系，因为一个特征值对应了很多个特征向量，它们都是线性相关的。